In [ ]:
import numpy as np
from tensorflow.keras.models import load_model

from utils.frame_extraction import extract_frames
from utils.preprocessing import preprocess_frames, reshape_for_sequence

# Load model ONCE (important for performance)
MODEL_PATH = "model/best_model.h5"
model = load_model(MODEL_PATH)


def predict_video(
    video_file,
    num_frames=50,
    frame_size=(224, 224),
    threshold=0.5
):
    """
    Runs full prediction pipeline on a video.

    Parameters
    ----------
    video_file : UploadedFile or str
        Streamlit uploaded video or file path
    num_frames : int
        Number of frames to sample
    frame_size : tuple
        Resize size for frames
    threshold : float
        Classification threshold

    Returns
    -------
    dict
        {
            "label": str,
            "probability": float
        }
    """

    # 1. Extract frames
    frames = extract_frames(
        video_file,
        num_frames=num_frames,
        target_size=frame_size
    )

    # 2. Preprocess
    frames = preprocess_frames(frames)

    # 3. Reshape for sequence model
    frames = reshape_for_sequence(frames)

    # 4. Predict
    preds = model.predict(frames, verbose=0)

    # Handle sigmoid or softmax output
    if preds.shape[-1] == 1:
        prob = float(preds[0][0])
    else:
        prob = float(preds[0][1])

    label = "Positive" if prob >= threshold else "Negative"

    return {
        "label": label,
        "probability": round(prob, 4)
    }
